# Logistic Regression Gradient & Convexity — Student Lab

Complete all TODOs. This lab is math-first and stability-first.

In [1]:
import numpy as np

def check(name: str, cond: bool):
    if not cond:
        raise AssertionError(f'Failed: {name}')
    print(f'OK: {name}')

rng = np.random.default_rng(0)

## Section 0 — Synthetic Dataset
We’ll create a binary classification dataset with both separable and non-separable regimes.

In [2]:
def make_data(n=400, d=5, separable=False):
    X = rng.standard_normal((n, d))
    w_true = rng.standard_normal(d)
    logits = X @ w_true
    if separable:
        y = (logits > 0).astype(int)
    else:
        # add noise so it's not perfectly separable
        logits = logits + 0.5 * rng.standard_normal(n)
        probs = 1 / (1 + np.exp(-logits))
        y = (rng.random(n) < probs).astype(int)
    return X, y, w_true

X, y, w_true = make_data(separable=False)
check('shapes', X.ndim==2 and y.ndim==1 and X.shape[0]==y.shape[0])
X.shape, y.mean()

OK: shapes


((400, 5), np.float64(0.515))

## Section 1 — Sigmoid + Stable Loss

### Task 1.1: Stable sigmoid

# TODO: implement a numerically stable sigmoid.
# HINT:
# - Use `np.where(z>=0, ...)` trick to avoid overflow.

**Explain:** Why does sigmoid saturate for large |z| and what does that do to gradients?

In [10]:
def sigmoid(z):
    # TODO
    z = np.asarray(z)
    x = 1 / (1 + np.exp(-z))
    y = np.exp(z) / (1 + np.exp(z))
    return np.where(z >= 0, x, y)

z = np.array([-1000.0, -10.0, 0.0, 10.0, 1000.0])
p = sigmoid(z)
print(p)
check('in_0_1', np.all((p >= 0) & (p <= 1)))
check('monotonic', np.all(np.diff(p) >= 0))

[0.00000000e+00 4.53978687e-05 5.00000000e-01 9.99954602e-01
 1.00000000e+00]
OK: in_0_1
OK: monotonic


/tmp/ipython-input-1819154982.py:4: RuntimeWarning: overflow encountered in exp
  x = 1 / (1 + np.exp(-z))
/tmp/ipython-input-1819154982.py:5: RuntimeWarning: overflow encountered in exp
  y = np.exp(z) / (1 + np.exp(z))
/tmp/ipython-input-1819154982.py:5: RuntimeWarning: invalid value encountered in divide
  y = np.exp(z) / (1 + np.exp(z))


### Task 1.2: Stable binary cross-entropy loss

We use labels y in {0,1}.

Loss per example: `-y log(p) - (1-y) log(1-p)` where p=sigmoid(z).

# TODO: implement stable loss without NaNs/inf.
# HINT:
# - Clip p to [eps, 1-eps]
# - Alternatively use softplus: log(1+exp(z))

**Interview Angle:** explain a stable form of log-loss.

In [11]:
def bce_loss(y, p):
    # TODO
    eps = 1e-12
    p = np.clip(p, eps, 1 - eps)
    y = y.astype(float)
    return float(-np.mean(y * np.log(p) + (1 - y)*np.log(1 - p)))

# sanity: loss is finite
z = X @ np.zeros(X.shape[1])
p = sigmoid(z)
L = bce_loss(y, p)
print('loss', L)
check('finite_loss', np.isfinite(L))

loss 0.6931471805599452
OK: finite_loss


## Section 2 — Gradient (Derive → Implement → Check)

### Task 2.1: Derive the gradient (write in markdown)
Show that for loss averaged over n examples:
`grad = X^T (p - y) / n`

**Checkpoint:** What is the shape of grad?

### Task 2.2: Implement loss and gradient

# TODO: implement `logreg_loss_and_grad(X, y, w)` returning (loss, grad).
# HINT:
# - z = X @ w
# - p = sigmoid(z)
# - loss = BCE
# - grad = X.T @ (p - y) / n

**FAANG gotcha:** ensure y is 0/1, not -1/+1.

In [17]:
def logreg_loss_and_grad(X, y, w):
    # TODO
    z = X @ w
    # p = 1 / (1 + np.exp(-z))
    p = sigmoid(z)
    loss = float(-np.mean(y * np.log(p) + (1 - y) * np.log(1 - p)))
    n = X.shape[0]
    grad = X.T @ (p - y) / n
    return loss, grad

w0 = rng.standard_normal(X.shape[1])
loss, grad = logreg_loss_and_grad(X, y, w0)
print('loss', loss, 'grad_norm', np.linalg.norm(grad))
check('grad_shape', grad.shape == w0.shape)
check('finite', np.isfinite(loss) and np.all(np.isfinite(grad)))

loss 1.0600908718928417 grad_norm 0.4480760385916178
OK: grad_shape
OK: finite


### Task 2.3: Numerical gradient check (finite differences)

# TODO: implement numerical gradient and compare to analytic grad.
# HINT: central difference

**Checkpoint:** Why can eps too small make the check worse?

In [ ]:
def numerical_grad(f, w, eps=1e-5):
    # TODO
    ...

f = lambda v: logreg_loss_and_grad(X, y, v)[0]
g_num = numerical_grad(f, w0)
_, g = logreg_loss_and_grad(X, y, w0)

max_abs = np.max(np.abs(g - g_num))
print('max_abs_diff', max_abs)
check('grad_check', max_abs < 1e-5)

## Section 3 — Convexity & Hessian Intuition

### Task 3.1: Hessian-vector product (HVP)

For logistic regression:
H = (1/n) X^T S X where S = diag(p(1-p)).

Implement HVP: compute H@v without building full H explicitly.

# HINT:
- s = p*(1-p)
- compute Xv then multiply by s then X^T


In [ ]:
def hvp(X, y, w, v):
    # TODO
    ...

v = rng.standard_normal(X.shape[1])
Hv = hvp(X, y, w0, v)
check('hvp_shape', Hv.shape == v.shape)
check('hvp_finite', np.all(np.isfinite(Hv)))

### Task 3.2: Empirical PSD check

Check that v^T H v >= 0 for random v (PSD).

**Explain:** Why does PSD imply convexity?

In [ ]:
def vHv(X, y, w, v):
    return float(v @ hvp(X, y, w, v))

vals = []
for _ in range(50):
    v = rng.standard_normal(X.shape[1])
    vals.append(vHv(X, y, w0, v))
print('min vHv', min(vals))
check('psd', min(vals) > -1e-8)

## Section 4 — Optimization (Bonus)

### Task 4.1: One step of GD vs Newton (conceptual)

Implement one gradient descent step. (Newton step is optional.)

**FAANG gotcha:** perfectly separable data can push weights to infinity; explain why.

In [ ]:
def gd_step(X, y, w, lr):
    # TODO
    ...

w1 = gd_step(X, y, w0, lr=0.1)
loss0, _ = logreg_loss_and_grad(X, y, w0)
loss1, _ = logreg_loss_and_grad(X, y, w1)
print('loss0', loss0, 'loss1', loss1)
check('decreased', loss1 <= loss0 + 1e-12)

---
## Submission Checklist
- All TODOs completed
- Gradient check passes
- PSD check passes
- Short answers to checkpoint questions
